# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
This is a classification problem, because you have to decide whether a certain student who will need early internvention, therefore classifying them as `'needing intervention'`.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Make matlab plot inline for data visualization
%matplotlib inline

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1]  - 1  # indicating the column `passed` which is not a feature

# TODO: Calculate passing students
n_passed = np.argwhere(student_data['passed'] == "yes").shape[0]

# TODO: Calculate failing students
n_failed = np.argwhere(student_data['passed'] == "no").shape[0]

# TODO: Calculate graduation rate
grad_rate = (float(n_passed) / float(n_passed + n_failed)) * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()
print y_all.head()

X_all = X_all.drop('romantic', 1)
X_all = X_all.drop('nursery', 1)


Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
                                

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (46 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'higher', 'internet', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn.model_selection import train_test_split


# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, train_size=num_train)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

#### SVM

- This algorithm (as many others) is currently used by a lot of companies on many tasks that involves supervised classification or regression. A real world application can vary from [classifying high resolution satellite image](https://infoscience.epfl.ch/record/63248/files/TPD_Karlsson.pdf), [object recognition for autonomous vehicles](http://cs229.stanford.edu/proj2015/170_report.pdf) to [text categorization](https://www.cs.cornell.edu/people/tj/publications/joachims_98a.pdf).
- SVM's are good for domains where there is a clear margin of separation within the data. Also, according to [this](https://www.cs.rutgers.edu/~mlittman/courses/ml04/svm.pdf) power point, SVM's are good because training is relatively easy compared to other models (such as neural networks), it can accept non-traditional data such as strings and trees (instead of feature vectors) and the trade-off between classifier complexity and error can be controlled explicitly.

- SVM's are not so good for large data sets because they don't perform so well, also, they are not very good in a very noisy dataset.

-  Another point that could be counted either as a weakness and as a strenght is the fact that you need a kernel function. It is a weakness because there is an extra layer of complexity needed in order to get the model generalizing well. But, it could also be seen as a strenght since it allows a way for the business logic to be embeded into the learning algorithm.

- In order to know which algorithms to use, I decided to simulate them all on the data at hand before diving into searching about each specific algorithm. While doing the tests I noticed that SVM's scored the best on the testing set, and had a pretty high trainig score as well when we were using 300 data points. In addition of that, I thought there could be a clear margin of separation within the data that could be found by SVM. Also, since the data is not large at all, SVM will tend to perform really well.


### Gaussian Naive Bayes

- One real world application of this algorithm is the classifications of documents as described on [ this article](http://www.ic.unicamp.br/~rocha/teaching/2011s2/mc906/aulas/naive-bayes-classifier.pdf). Also, there are people who use this technique to [filter spam](https://en.wikipedia.org/wiki/Naive_Bayes_spam_filtering). 

- This is a simple algorithm that generalizes well to cases where you don't have an enormous amount of data. It will also perform well even if the Naive Bayes assumption doesn't hold, and it will perform even better if it does.

- Its main disadvantage is that it can't interact between features. Therefore, it wouldn't be able to learn that even though a student that had problems at home or didn't have internet werent likely to need guidance, a student with both would.

- This model was chosen only after I plotted the learning curves for all the algorithms, and noticed that the Gaussian Naive Bayes was the only one whose curve started low when dealing with 100 datapoints, but gradually grew from a score of 0.4 to about ~0.75 as the number of points grew. Also, in addition to generalizing better to a larger dataset, this model seemed to reduce variance, since both testing and training curves tended to approach each other. In addition of that, I wanted to investigate how a model that does not take into consideration the relationship between features would behave on this dataset, to find out if the relationship between the features mattered in this case.



### K-Nearest Neighbors

- One real world application of knn is [facial recognition](http://users.cecs.anu.edu.au/~ramtin/ICSPCS/ICSPCS'07/papers/198.pdf).

- KNN deals nicely with noisy training data and is effective if the training data is large. The cost of the learning process is zero, since you will get the nearest neighbours to the new instance point. 

- KNN tends to have a lot of bias through locality (near points are similar), smoothness (averaging) and equally weighting each feature.

- In addition to the points menitoned above, KNN also has a point that can be considered as both, weakness and strength. Which is the fact that you need to compute a distance function, in which case, it increases the complexity of the model to generalize well to the data. But also, gives the opportunity to compute the business logic related to the problem itself. 

- KNN was also chosen due to the analyisis done by plotting the learning curves of each of the algorithms listed below. When analyzing the curve, it seemed to maintain a good distance between the training and testing curves throughout all different size variations of the data points. Also, after plotting the complexity of a few models, I could notice that if `k` of KNN was changed to 30, it would generalize better and produce some of the best results of all the models tested. In addition of that, I also wanted to see if the similarities found between the features had a high impact on the model's results after all, and since KNN largely rely on the similarities between features to make its predictions, I thought it would be a good candidate for this problem.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
# TODO: Import any additional functionality you may need here
from sklearn.learning_curve import learning_curve
from sklearn.cross_validation import ShuffleSplit, train_test_split
import copy as cp
import matplotlib.pyplot as plt

def split_data_train_test(X_all, y_all, num_train):
    num_test = X_all.shape[0] - num_train
    return train_test_split(X_all, y_all, test_size=num_test, train_size=num_train)

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target, name):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made " + name + " predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf['clf'].__class__.__name__, len(X_train))
    # Train the classifier
    train_classifier(clf['clf'], X_train, y_train)
    
    train_prediction = predict_labels(clf['clf'], X_train, y_train, 'train')
    test_prediction = predict_labels(clf['clf'], X_test, y_test, 'test')
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(train_prediction)
    print "F1 score for test set: {:.4f}.".format(test_prediction)
    
    return {
        'train_prediction': train_prediction,
        'test_prediction': test_prediction,
        'train_size' : X_train.shape[0],
        'name': clf['name']
    }
    
def run_prediction(clfs, X_train, y_train, X_test, y_test):
    X_train_100, x_test_100, y_train_100, y_test_100 =  split_data_train_test(X_all, y_all, 100)
    X_train_200, x_test_200, y_train_200, y_test_200 =  split_data_train_test(X_all, y_all, 200)
    
    train_size_variations = [
        (X_train_100, y_train_100, x_test_100, y_test_100),
        (X_train_200, y_train_200, x_test_200, y_test_200),
        (X_train, y_train, X_test, y_test)
    ]
    
    predictions = []
    for i in range(len(clfs)):
        for idx in range(3):
            clf = cp.copy(clfs[i])
            prediction = train_predict(
                clf, 
                train_size_variations[idx][0],
                train_size_variations[idx][1],
                train_size_variations[idx][2],
                train_size_variations[idx][3]
            )
            
            predictions.append(prediction)
            print '\n--------------------\n'
        print '\n--------------------\n'  
        print '\n--------------------\n'
    return predictions


def table_print(data):
    # Extract the data from the predictions array of hashes
    table_predictions = [[x['name'], x['train_size'], float(x['train_prediction']), float(x['test_prediction'])] for x in data]

    # Sort it pased on `test_prediction`
    sorted_predictions = sorted(table_predictions, key=lambda row: row[3], reverse=True)

    # Import `DataFrame` from pandas
    from pandas import DataFrame

    # Declare cols names and DataFrame with the predictions and name of the classifiers
    # to print and analyze
    cols = ['Classifier name', 'Train Size', 'Train Prediction', 'Test Prediction']
    pred_dataframe = DataFrame(data=sorted_predictions, columns=cols)
    print pred_dataframe
    
# Function taken from http://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=[100, 200, 300]):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Testing score")

    plt.legend(loc="best")
    return plt
    
from sklearn.model_selection import validation_curve

def ModelComplexity(CLF ,Xx, yy, title, param_name, param_range):
    """ Calculates the performance of the model as model complexity increases.
        The learning and testing errors rates are then plotted. """
    
    # make a copy of params (avoid mutability)
    import copy as cp
    X = cp.copy(Xx)
    y = cp.copy(yy)
    clf = cp.copy(CLF)
    # Create 10 cross-validation sets for training and testing
    cv = ShuffleSplit(X.shape[0], n_iter = 10, test_size = 0.2, random_state = 0)
    
    # Calculate the training and testing scores
    train_scores, test_scores = validation_curve(
        clf,
        X,
        y,
        param_name = param_name, 
        param_range = param_range, 
        cv = cv
    )

    # Find the mean and standard deviation for smoothing
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)

    # Plot the validation curve
    plt.figure(figsize=(7, 5))
    plt.title(title +' Complexity Performance')
    plt.plot(param_range, train_mean, 'o-', color = 'r', label = 'Training Score')
    plt.plot(param_range, test_mean, 'o-', color = 'g', label = 'Testing Score')
    plt.fill_between(
        param_range, 
        train_mean - train_std,
        train_mean + train_std,
        alpha = 0.15,
        color = 'r'
    )
    plt.fill_between(
        param_range,
        test_mean - test_std,
        test_mean + test_std,
        alpha = 0.15,
        color = 'g'
    )
    
    # Visual aesthetics
    plt.legend(loc = 'lower right')
    plt.xlabel(param_name)
    plt.ylabel('Score')
    plt.ylim([-0.05,1.05])
    plt.show()



/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [ ]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import ExtraTreeClassifier, DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

# from sklearn import model_C

# TODO: Initialize the three models
clf_A = { 
    'id': 'mlnp',
    'clf' : MLPClassifier(), 
    'name': 'Multi Layer Neural Network' 
}
kernel = 'rbf'
clf_B = {
    'id': 'svm_rbf', 
    'clf' : svm.SVC(kernel=kernel, random_state=42), 
    'name': 'SVM SVC kernel=' + kernel 
}
clf_C = { 
    'id' : 'adaboost_dt',
    'clf' : AdaBoostClassifier(),
    'name': 'AbaBoost with DT' 
}
clf_D = { 
    'id' : 'gnb',
    'clf' : GaussianNB(), 
    'name': 'Gaussian Naive Bayes' 
}
k = 30
clf_E = { 
    'id' : 'knn',
    'clf' : KNeighborsClassifier(n_neighbors=k), 
    'name': 'K-nearest neighbors k='+ str(k) 
}
clf_F = {
    'id' : 'linear_svm_svc',
    'clf' : svm.LinearSVC(),
    'name' : 'SVM - Linear SVC'
}

clf_G = { 
    'id' : 'logreg',
    'clf' : LogisticRegression(), 
    'name': 'Logistic Regression' 
}
clf_H = { 
    'id' : 'sgd',
    'clf' : SGDClassifier(loss="hinge", penalty="elasticnet"), 
    'name': 'Stochastic Gradient Descent' 
}
clf_I = { 
    'id' : 'dt',
    'clf' : DecisionTreeClassifier(), 
    'name': 'DecisionTreeClassifier()' 
}


clfs = [
    clf_A,
    clf_B,
    clf_C,
    clf_D,
    clf_E,
    clf_F,
    clf_G,
    clf_H,
    clf_I
]

# Run multiple classifiers at the same time
#predictions = run_prediction(clfs, X_train, y_train, X_test, y_test)


# Print the dataframe to vizualize the predictions from all different classifiers sorted by the 
# test results
#table_print(predictions)


# ------------------
# CHOSEN CLASSIFIERS
# ------------------
svc_svm_rbf = clf_B
k_nearest_neighbours = clf_E
gaussian_naive_bayes = clf_D
chosen_clfs = [svc_svm_rbf, k_nearest_neighbours, gaussian_naive_bayes]
chosen_clfs_preds = run_prediction(chosen_clfs, X_train, y_train, X_test, y_test)
clfs = chosen_clfs

r = np.arange(1.0,10.1, 1)
coef0 = np.arange(0.5,1.1, 0.1)
n_neighbours = np.arange(1.0,100.1,10)
n_estimators = np.arange(1,500,100)
C= np.arange(0.1,1.1, 0.1)

# ----------------
# EXPERIMENTATION
# ----------------

## Plot Model Complexity
def plot_model_complexity():
    for clf in clfs:
        if clf['id'] == 'svm_rbf':
            ModelComplexity(clf['clf'], X_all, y_all, clf['name'], 'C',coef0)
        if clf['id'] == 'mlnp':
            ModelComplexity(clf['clf'], X_all, y_all, clf['name'], 'alpha',r)
        if clf['id'] == 'knn':
            ModelComplexity(clf['clf'], X_all, y_all, clf['name'], 'n_neighbors', n_neighbours)
        if clf['id'] == 'adaboost_dt':
            ModelComplexity(clf['clf'], X_all, y_all, clf['name'], 'n_estimators', n_estimators)
        if clf['id'] == 'logreg':
            ModelComplexity(clf['clf'], X_all, y_all, clf['name'], 'C', C)
        if clf['id'] == 'linear_svm_svc':
            ModelComplexity(clf['clf'], X_all, y_all, clf['name'], 'C', r)

def plot_learning_curves():
    for clf in clfs:
        plot_learning_curve(clf['clf'], clf['name'], X_all, y_all, ylim=(0, 1.01), cv=10, n_jobs=1)
        plt.show()
        
# ----------------
# Model Complexity
# ----------------  
#plot_model_complexity()

# ----------------
# Learning Curve
# ----------------
#plot_learning_curves()


Training a SVC using a training set size of 100. . .
Trained model in 0.0013 seconds
Made train predictions in 0.0006 seconds.
Made test predictions in 0.0015 seconds.
F1 score for training set: 0.8608.
F1 score for test set: 0.8117.

--------------------

Training a SVC using a training set size of 200. . .
Trained model in 0.0025 seconds
Made train predictions in 0.0018 seconds.
Made test predictions in 0.0018 seconds.
F1 score for training set: 0.8434.
F1 score for test set: 0.7923.

--------------------

Training a SVC using a training set size of 300. . .
Trained model in 0.0049 seconds
Made train predictions in 0.0038 seconds.
Made test predictions in 0.0015 seconds.
F1 score for training set: 0.8683.
F1 score for test set: 0.8079.

--------------------


--------------------


--------------------

Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0005 seconds
Made train predictions in 0.0019 seconds.
Made test predictions in 0.0031 seconds

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - SVM SVC**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0020          |       0.0023          |       0.9299      | 0.7932            |
| 200               |        0.0028          |       0.0019            |     0.8635      |  0.7935               |
| 300               |        0.0047           |      0.0019           |      0.8979     |   0.7273      |

** Classifer 2 - KNeighborsClassifier k=30**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0005             | 0.0025                  |  0.8166                |  0.8000              |
| 200               | 0.0005             | 0.0024                  |  0.8555                |  0.7619               |
| 300               | 0.0006             |   0.0023                |   0.8216              | 0.7808        |

** Classifer 3 - GaussianNB**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0008                  |   0.0003               | 0.8690          | 0.7389     |
| 200               |  0.0009                 |   0.0004               | 0.7757           | 0.7059                |
| 300               |  0.0011                 |   0.0004               | 0.8085           | 0.7273                |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

Based on the experimentation done earlier, it is quite clear that among the 3 algorithms **knn** have have shown the best performance, where mostly training scores are higher than testing, but they still do not differ by a large amount. Also, although it is the most costly whe making new predictions, it does not scale the training time as the data becomes larger.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

This model will solve the problem by finding `k` students that have similar features to a given student (whose you want to classify as needing help or not) and then finding out what are the classification most present on this group of `k` students. 

In order for this model to work well, there needs to be a function that will calculate the distance between the new student and the students in its surroundings, because its with this that we can embed to notion of similarity into the algorithm.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [ ]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV

# TODO: Create the parameters list you wish to tune
parameters = {
    'n_neighbors' : np.arange(1,100,10),
    'weights' : ['uniform', 'distance'],
    'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size' :  np.arange(30,120,30),
    'metric' : ['minkowski', 'manhattan', 'euclidean'],
    'p': [1, 2, 3]
}

# TODO: Initialize the classifier
clf = KNeighborsClassifier()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train, 'train'))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test, 'test'))

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [ ]:
print grid_obj.best_params_
# => {
#     'n_neighbors': 21, 
#     'algorithm': 'auto', 
#     'metric': 'minkowski', 
#     'p': 1, 
#     'weights': 'uniform', 
#     'leaf_size': 60
#    }

### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

The final F1 score for training was **0.8238** and **0.7922** for testing. While the training score is not as good as the untuned model, the testing score has improved by a factor of nearly 1%. This may be due to the fact that I had already tuned the models earlier by computing the complexity of them, therefore being able to perceive best params before doing grid search. Also, it is important to note that both scores got closer to each other, indicating a lower variance, but danger of higher bias.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.